In [1]:
import numpy as np
import pandas as pd
from flask import Flask, render_template, request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json
import bs4 as bs
import urllib.request
import pickle
import requests

In [2]:

data = pd.read_csv('main_data.csv')
# creating a count matrix
cv = CountVectorizer()
count_matrix = cv.fit_transform(data['comb'])
# creating a similarity score matrix
similarity = cosine_similarity(count_matrix)


In [10]:
data

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6005,Joseph Kosinski,Tom Cruise,Miles Teller,Jennifer Connelly,Action Drama,top gun: maverick,Tom Cruise Miles Teller Jennifer Connelly Jose...
6006,Joel Crawford,Nicolas Cage,Emma Stone,Ryan Reynolds,Animation Adventure Family,the croods 2,Nicolas Cage Emma Stone Ryan Reynolds Joel Cra...
6007,Liesl Tommy,Jennifer Hudson,Forest Whitaker,Marlon Wayans,Music Drama,respect,Jennifer Hudson Forest Whitaker Marlon Wayans ...
6008,Ridley Scott,Matt Damon,Adam Driver,Jodie Comer,Drama,the last duel,Matt Damon Adam Driver Jodie Comer Ridley Scot...


In [11]:
similarity

array([[1.        , 0.24174689, 0.16116459, ..., 0.        , 0.        ,
        0.        ],
       [0.24174689, 1.        , 0.18181818, ..., 0.        , 0.        ,
        0.        ],
       [0.16116459, 0.18181818, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.10540926,
        0.1       ],
       [0.        , 0.        , 0.        , ..., 0.10540926, 1.        ,
        0.10540926],
       [0.        , 0.        , 0.        , ..., 0.1       , 0.10540926,
        1.        ]])

In [14]:
with open('ratings.txt') as f:
    lines = f.read().splitlines()

In [19]:
lines[0]

'[3471, 0.4629100498862758]'

In [56]:
with open('rating', 'rb') as fp:
    n_list = pickle.load(fp)
    n_list = round(n_list[1],5)
n_list

1.0

In [23]:
n_list[1]

0.4629100498862758

In [28]:
data['movie_title'][1584]

'little shop of horrors'

In [47]:
with open('static/ratings', 'rb') as fp:
    n_list = pickle.load(fp)
    n_list = [round(n_list[1],5) for n_list in n_list]
n_list

[0.43519,
 0.36364,
 0.36364,
 0.35533,
 0.34816,
 0.34816,
 0.34816,
 0.34816,
 0.3345,
 0.3345]

In [46]:
with open('static/ratings', 'rb') as fp:
    n_list = pickle.load(fp)
n_list

[(4311, 0.4351941398892446),
 (3, 0.36363636363636365),
 (1111, 0.36363636363636365),
 (1818, 0.3553345272593508),
 (57, 0.3481553119113957),
 (1416, 0.3481553119113957),
 (1462, 0.3481553119113957),
 (3881, 0.3481553119113957),
 (822, 0.3344968040028363),
 (1113, 0.3344968040028363)]

In [41]:
filename = 'nlp_model.pkl'
clf = pickle.load(open(filename, 'rb'))
vectorizer = pickle.load(open('tranform.pkl','rb'))

c:\Users\Ghost\anaconda3\envs\movie_project\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.23.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Ghost\AppData\Local\Temp\ipykernel_12552\121990886.py:3: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  vectorizer = pickle.load(open('tranform.pkl','rb'))
c:\Users\Ghost\anaconda3\envs\movie_project\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#secu

In [71]:
text = " Hey this movie is bad."
text_array = np.array([text])
vect = vectorizer.transform(text_array)
tf_idf_value = vect.toarray()
tf_idf_key = vectorizer.get_feature_names_out()

In [72]:
tf_idf_dict = {}
tf_idf = []
for i in range(len(tf_idf_key)):
    tf_idf_dict[tf_idf_key[i]] = tf_idf_value[0][i]


In [77]:
from itertools import islice
tf_idf_dict
my_dict = {'a': 5, 'b': 2, 'c': 10, 'd': 1}

sorted_dict = dict(sorted(tf_idf_dict.items(), key=lambda x: x[1],reverse=True))
sliced_dict = dict(islice(sorted_dict.items(),20))
print(sliced_dict)

{'hey': 0.6818806870803984, 'bad': 0.671534394266108, 'movie': 0.2899660081878823, '00': 0.0, '007': 0.0, '10': 0.0, '10pm': 0.0, '12': 0.0, '16': 0.0, '17': 0.0, '1984': 0.0, '1st': 0.0, '200': 0.0, '2007': 0.0, '286': 0.0, '2nd': 0.0, '30': 0.0, '31st': 0.0, '33': 0.0, '3333': 0.0}


In [67]:
reviews_list = []
reviews_list.append(text)

In [70]:
my_dict = {'a': 5, 'b': 2, 'c': 10, 'd': 1}

sorted_dict = dict(sorted(my_dict.items(), key=lambda x: x[1],reverse=True))

print(sorted_dict)

{'c': 10, 'a': 5, 'b': 2, 'd': 1}


In [59]:
test_dict

{0: {'hey', 'nice'}}

In [78]:
vectorizer.get_feature_names_out()

array(['00', '007', '10', ..., 'yuh', 'zach', 'zen'], dtype=object)